This is a text generation example using RNN units with LSTM.  the example is from [this github page](https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py).



In [1]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
print()

Using TensorFlow backend.


606208/600901 [==============================] - 1s 2us/step
corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
200285/200285 [==============================] - 435s 2ms/step - loss: 1.9952

----- diversity: 0.2
----- Generating with seed: "ll the high school of taste; but
one mus"
ll the high school of taste; but
one must the sense of the still the senses of the still the strengure of the still and the sense there a subjection of the still and the sense of the subjent of the sensual and in the sense of the subjert the son and the subjert and in the strenger and and in the stand the subjent of the such a man in the stand of the sensifice the stand and in the sense of the senses of the most in the sensulity and in 
----- diversity: 0.5
----- Generating with seed: "ll the high school of taste; but
one mus"
ll the high school of taste; but
one must the most most pacted and the

when people have lived long together underch is man as the fact of the divil in the order to the explainer, and it is does not knowlesth--and a proces to the insiph and free devils to the believes to childs and present--who also has to believes its such as the power in the proprior and best as a all the explooration of the all and intellectual and problem of the exist to the self the complaces and ones of the subject of the same to the 
----- diversity: 1.0
----- Generating with seed: "when people have lived long together und"
when people have lived long together under worl;
and he
is lid
of bounted-oblearn the animal honoursely our oul unisever
list reprobatidne more time
offinsivem--wroth a much there in the slave evil as beatnelist!ouh of may--al-adajly so and uneuroum and have coindted, has tooght been too who kind sotter. their troubreper
refunements, his etiffests of not history--rue
banle, who  equally from "pilsot to the can is to imsruip is to the pr
----- diversity: 1.2
-----

which protmoss acclesed he bleve.fuls call we ver-what
itjence generately
more pat who s quise of utiling, where no manters, a scrople.. this ; pasne from
anikiok ar? "froolves is matter 1
--------------------------------------------------
Iteration 8
Epoch 1/1
200285/200285 [==============================] - 402s 2ms/step - loss: 1.4359

----- diversity: 0.2
----- Generating with seed: " less beautiful but
harder souls, that i"
 less beautiful but
harder souls, that is the thing and and the thing the pains of the considere the standard the standard the scientificate of the hig

/home/joe/test/lib/python3.5/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log


...

lf contempt, that is by a false and unscelsatsthatointattelatoheantu a thaso rt i aostleistlus e nere las an e tei t asits is  hi ste ut_ihtrr sliths aoetaatt e tosloa  tir s o s ioe nl t  esluetiths a the teatheins ietheanenrha s te  t aser inseshthtine ses   i eaes  othlsaau n t  th thl nees iue ense i sattth rel tho initelsileisat tilhthe se tit ten letshth l ass ththeat eethar te ers naltsuse    ae he  ostit thhe  to  iutr iseit the
----- diversity: 1.2
----- Generating with seed: "lf contempt, that is by a false and unsc"
lf contempt, that is by a false and unscoit it eseoit th in e esra  ane ertse aathitsioeshe lise aan  ohthels htlr  e islssaltrsto   n ustiai orl hoes aoselotelie oroareol e anrrutisthe s on ase ut i lersitti sse i  so t ienttot ira si ea lahhoi alee ttaol rathlulheiroitsthre ee tsileenlatalleronhiss sreirsoallt entorli thelo siahe tstaterielti  aiasenheanuar tluttiean nsneinle lseie olittlaiesss aarern ers  thera eslthhle nlah  s hhi t
--------------------------

dispense with it. science, however, reco an  ae  tn  oe hen e  t aen  n ne te o  ae  ane aate  o  e a   the  heth  aa an  tthe a  al ee  a t ane the oee  the  ane  aee   ane e oe inoe the at ee t a as an   ou  an al ann ou   t n  oinhe t a ae ane  ina   oeth ae  on ae he e  the   i thean  t an a  oe ane  ther ean aann t i  te  ae n a  than  oe  a a  a ao the an  alan an e  an thie  an an san an a a  teo e ae the a thh   g    o i  aa  f 
----- diversity: 0.5
----- Generating with seed: "dispense with it. science, however, reco"
dispense with it. science, however, reco thno n he lnt uen te in ao s an aesh iaouhot  o heeee as th al  e athaae et aa itne est nroauu ata ot aa( oane an  hel
se =o un  on o eitit totre eose t th anae1s en an see an  an on aandu i has noote ane t inon e a ao u t  t  a e i teaosherend   t th a l tero thi se ri o aue tht et d r ae u aree  anths u as a uoenle at o an  ae t e  annellu tuthae q a t m he t th   i henet au toe o  s  iontinh t
----- diversity: 1.0
-----

select, more refinheoeses e-t t   t a o ea  r il tne sus atheriteiero annanes oothis  ainno  ieroteoniuhnu a niearoosn hes s  to t neesua rehh h se in a aratr at ieau snh it th ouianouth tinoun i teratto sst osesooee  hasis nitinanon  tnoeereoie rranet enoioth senias sha is  tieesai een sh atsr lhent hieoseiioeabo ns hnn aeeetseriisis oorentash tn rite rort ol he ahe t ais t tiitnen  tsioatosh a seeauheoahteis ol l
----- diversity: 1.2
----- Generating with seed: "e advantage; the more
select, more refin"
e advantage; the more
select, more refinhtlrearhia rhioisen ne auano igeoreat   tene nriirthoseaisn iienoon iiioten rn suotressit sar ioetntieas a totinss enre u o eesel hif thoiliirnithneeonol h  t oo ees ouea etleril e atuarsesin ausiooae  lhis,visererarrn  aonoevr i hl h  soltoth tho se rinlrs iial es lu iueurhehitth  os i enee iniorrooenitietilniioesieensrirsothsotlesutuanor son tr lue ettioil iere aneens suss sh oeonintur nntflluio
------------------------------------------------

er certain circumstances, which made hime afrr rere the e t  ane the ih  ote ( aout tr all an  tn t ot t  on otr ane te the  hmee the  t ae  se an an ans tn in the a  no thee qn at tre an  t th ans an an  inu thirere   th thent  hal  ane an ou tohe thhuth ahh  tha an the es in fat l a  thi aas th0 th inn n te in oin  or ornt the the inn  a n al an  t thes alln on ae ant  on al ao the  ah i thi aree therese ar an  hh thh  at ant alel ier
----- diversity: 0.5
----- Generating with seed: "er certain circumstances, which made him"
er certain circumstances, which made himllato s sothtan er anere o in ooe anti  tharie s oe ali  ane suol an  e ale sne ahes ahei    rt in  nh aiiniee oo ee trrebeinthens alin ithnehe a u,h u o n anre an oll ouie e a tere thite s  there se e erer al ao,e onn an t e s t rge ale iaot ion a o iin in  at tis  ohsehahal alonrtthjn onars il arhunanarenh oe hot  tnaean ottethas theoe  otou ala the hai o a- nix reret hna  e th  thltl t al orlin
----- diversity: 1.0
-----

only world we know are us tu te hhunr oisointest ali t s nesilole s as eh ssse ihtheasrt fen rieoheutoei tholohereeen anasit  atloe otree niheanu leeson areseu  oe  ia alaot ahri thstta(an aarshaol sh   ahtitht ertsnn ns s th to si  oot sss s isieo hr tonaailheeisno us siaisooosusre lioneauoa ite a torrasaehhi hetlih iues aner"eroohrritnt to there ha tin  touae  a itl arintihrsoeea tth ioehesnsiotithhi  aareaonutnsen e e 
----- diversity: 1.2
----- Generating with seed: "nations of the
only world we know are us"
nations of the
only world we know are ussuatse etlae nse h iseosahnlenasner u   lnshiois  ho entt ton-e hieeh etr till aieathosietelse  hs ennelleees ts in r tasiauaie u nie ttha  ltathhes th ae eh  erqutnnereer rener as ilas taroeo aa a tr le anae oal ns  oaatiso th eonshralhelkats a tius sssa an o lethshe  uunerun st iha theeineshte arorosaesua rtoloo nenaithen ae sn onsa s jahie th  rt noe elesarleausirns t a r isnlnna,oht tet ttei  
-----------------------------------------